In [1]:
from utils import *
from fn_test import _test_overlapping
import glob
import pandas as pd
import json

## Tags and save nested entities.

In [2]:
def tags_nested_ner(path_raw_data, path_save, tag, group, max_levels, engine):
    path_save_nested     = path_save+'/nested_ner/'+tag+'/'
    path_save_topl       = path_save+'/flatten_ner/'+tag+'/'

    ### Find raw datas.
    filenames_csv        = [f for f in glob.glob(path_raw_data+'**/*.csv', recursive=True)]
    filename_json        = [f for f in glob.glob(path_raw_data+'**/*.json', recursive=True)]

    # Show all files to download
#     print('\n-- Input paths --')
#     print('CSV files')
#     [print(f) for f in filenames_csv]

#     print('\nJson files')
#     [print(f) for f in filename_json]

#     print('\n-- Output paths --')
#     print('Output path: nested_ner  : ', path_save_topl)
#     print('Output path: flatten_ner : ', path_save_topl)

    ### Tags nested ner ###
    temp_sentences   = []
    temp_overlappint = {}
    for filen_csv, filen_json in zip(filenames_csv, filename_json):
        overlap_en       = []
        # Check the file name to load and save.
        checklotdir_csv   = filen_csv.split('/')[-2]
        checklotdir_json  = filen_json.split('/')[-2]
        save_file_name    = filen_json.split('/')[-2]

        assert checklotdir_csv == checklotdir_json
        print('\n',checklotdir_csv, checklotdir_json)
        print('{}'.format(filen_csv.split('/')[-1]))
        print('{}'.format( filen_json.split('/')[-1]))

        ## Create output files
        sftags_nested_ner = open(path_save_nested+save_file_name+'_nested.data', 'w')

        ## Load CSV file
        data_csv  = pd.read_csv(filen_csv)

        ## Load Json file 
        read_file = open(filen_json, "r")
        data_json = json.load(read_file)

        ## Tags entities
        for idx, datas in enumerate(zip(data_json, data_csv['ssg'], data_csv['text_clean'])):

            ## Remove pipe
            datas = remove_pipe(datas[0],datas[1],datas[2])

            ## Prepare data
            entities            = datas['entities']
            text                = datas['text']
            token_idx_en        = get_all_entities_index(entities)
            words_token         = token_en2words(token_idx_en, text, engine=engine)
            words_tags_nested_  = tags_nested(words_token, entities, max_levels, group=group)
            words_tags_nested   = tags_bioes(words_tags_nested_, max_levels)
            
            ## Test
            #Check overlappint entities:
            if(_test_overlapping(words_tags_nested_)):
                #print('Overlapping:',idx)
                overlap_en.append(idx)
                continue
                
            if(_check_tokenizer(text, words_token)):
                print('Tokenizer error',idx)
                raise idx
            
            if(False):
                _show_details(datas, words_tags_nested_)
                return 0
            
            ## Save nested corpus ##
            save_data(words_tags_nested, sftags_nested_ner, nested=True)
            temp_sentences.append(words_tags_nested)
        sftags_nested_ner.close()
        
        # Report error
        temp_overlappint[str(save_file_name)] = [str(x) for x in overlap_en]
        
    ### Save flatten ner(Top-level) ###
    srcf = open(path_save_topl+'data.src', 'w')
    trgf = open(path_save_topl+'data.trg', 'w')

    for sent in temp_sentences:
        source = [token[0] for token in sent]
        target = [token[1] for token in sent]
        srcf.writelines('|'.join(source)+'\n')
        trgf.writelines('|'.join(target)+'\n')
    srcf.close()
    trgf.close()
    
    print('\nOverlapping sents no. :',temp_overlappint)

In [3]:
def _check_tokenizer(text, words_token):
    all_chars_len  = len(text)
    all_tokens_len = np.sum([len(x) for x in words_token])
#     print(all_chars_len)
#     print(all_tokens_len, words_token)
    assert all_chars_len == all_tokens_len

def _show_details(datas, words_tags_nested_):
    check_sentence(datas)
    [print(s) for s in words_tags_nested_]
    return 0

In [4]:
%%time
# This path for loading raw data from the VISTEC nested ner corpus.
path_corpus               = '/home/module/data/vistec_corpus/NER_dataset_20200629/'
path_save                 = '/home/module/data/vistec_newmm4L'
# path_save                 = './results/test2'

max_levels                = 8
engine                    = "newmm"  # ['newmm', 'longest', 'deepcut', 'icu', 'ulmfit', 'attacut']
create_save_dir(path_save)

if __name__ == '__main__':
    tags_nested_ner(path_corpus, path_save, 'maintags', True, max_levels, engine)
    tags_nested_ner(path_corpus, path_save, 'subtags', False, max_levels, engine)


 datalot1 datalot1
ner_lot1_050620_num_word_192449_num_tag_21688.csv
ner_lot1_050620_num_word_192449_num_tag_21688.json

 datalot2 datalot2
ner_lot2_050620_num_word_195530_num_tag_32232.csv
ner_lot2_050620_num_word_195530_num_tag_32232.json

 datalot3 datalot3
ner_lot3_050620_num_word_198387_num_tag_34144.csv
ner_lot3_050620_num_word_198387_num_tag_34144.json

 datalot4 datalot4
ner_lot4_050620_num_word_401684_num_tag_70076.csv
ner_lot4_050620_num_word_401684_num_tag_70076.json

Overlapping sents no. : {'datalot1': ['151', '185', '261', '307', '317', '321', '324', '461', '514', '518', '519', '520', '522', '524', '525', '526', '527', '528', '532', '533', '535', '554', '556', '558', '559', '561', '563', '564', '573', '574', '575', '577', '581', '624', '625', '643', '656', '672', '683', '687', '690', '695', '701', '711', '714', '723', '726', '791'], 'datalot2': ['12', '17', '39', '51', '55', '74', '76', '77', '78', '81', '92', '124', '126', '134', '138', '151', '152', '153', '157', '191'

In [5]:
# 90 sents
# attacut : 1min 22s
# newmm   : 4.35 s

# all
# attacut : 28min 44s

In [6]:
def _check_tokenizer(text, words_token):
    all_chars_len  = len(text)
    all_tokens_len = np.sum([len(x) for x in words_token])
#     print([len(x) for x in words_token])
#     print(all_chars_len)
#     print(all_tokens_len, words_token)
    assert all_chars_len == all_tokens_len

In [7]:
from pythainlp.tokenize import word_tokenize
text = "เรา รักภาษาบ้านเกิด234"

In [8]:
icu_token = word_tokenize(text, engine="icu")
icu_token

['เรา', ' ', 'รัก', 'ภาษา', 'บ้าน', 'เกิด', ' ', '234', ' ']

In [9]:
attacut_token = word_tokenize(text, engine="attacut")
attacut_token

['เรา', ' ', 'รัก', 'ภาษา', 'บ้าน', 'เกิด', '234']

In [10]:
newmm_token = word_tokenize(text, engine="newmm")
newmm_token

['เรา', ' ', 'รัก', 'ภาษา', 'บ้านเกิด', '234']

In [11]:
_check_tokenizer(text,newmm_token)

In [12]:
{
 'dataLot1': ['151', '185', '261', '307', '317', '321', '324', '461', '514', '518', '519', '520', '522', '524', '525', '526', '527', '528', '532', '533', '535', '554', '556', '558', '559', '561', '563', '564', '573', '574', '575', '577', '581', '624', '625', '643', '656', '672', '683', '687', '690', '695', '701', '711', '714', '723', '726', '791'], 
 'datalot2': ['12', '17', '39', '51', '55', '74', '76', '77', '78', '81', '92', '124', '126', '134', '138', '151', '152', '153', '157', '191', '198', '215', '219', '237', '282', '308', '318', '350', '388', '391', '393', '394', '400', '403', '407', '410', '411', '415', '416', '420', '425', '428', '464', '475', '506', '593'], 
 'datalot3': ['23', '25', '49', '65', '67', '110', '116', '118', '120', '125', '138', '166', '183', '184', '207', '248', '249', '250', '255', '263', '274', '275', '278', '279', '280', '284', '286', '323', '325', '336', '339', '348', '388', '389', '391', '417', '418', '419', '421', '423', '424', '425', '427', '428', '429', '432', '433', '434', '435', '437', '440', '443', '449', '455', '467', '477', '495', '499', '500', '509', '511', '514', '519', '524', '542', '544']
}
pass

In [15]:
overlapping = {'datalot1': ['151', '185', '261', '307', '317', '321', '324', '461', '514', '518', '519', '520', '522', '524', '525', '526', '527', '528', '532', '533', '535', '554', '556', '558', '559', '561', '563', '564', '573', '574', '575', '577', '581', '624', '625', '643', '656', '672', '683', '687', '690', '695', '701', '711', '714', '723', '726', '791'], 'datalot2': ['12', '17', '39', '51', '55', '74', '76', '77', '78', '81', '92', '124', '126', '134', '138', '151', '152', '153', '157', '191', '198', '215', '219', '237', '282', '308', '318', '350', '388', '391', '393', '394', '400', '403', '407', '410', '411', '415', '416', '420', '425', '428', '464', '475', '506', '593'], 'datalot3': ['23', '25', '49', '65', '67', '110', '116', '118', '120', '125', '138', '166', '183', '184', '207', '248', '249', '250', '255', '263', '274', '275', '278', '279', '280', '284', '286', '323', '325', '336', '339', '348', '388', '389', '391', '417', '418', '419', '421', '423', '424', '425', '427', '428', '429', '432', '433', '434', '435', '437', '440', '443', '444', '445', '446', '448', '449', '455', '467', '477', '495', '499', '500', '509', '511', '514', '519', '524', '542', '544', '556', '560', '568', '572'], 'datalot4': ['10', '65', '74', '80', '84', '92', '98', '99', '100', '101', '110', '155', '169', '171', '188', '243', '249', '251', '258', '264', '265', '288', '435', '437', '438', '467', '491', '505', '516', '569', '577', '578', '601', '619', '620', '621', '624', '640', '651', '658', '666', '667', '668', '726', '734', '735', '740', '752', '812', '813', '817', '818', '819', '821', '822', '833', '841', '843', '844', '849', '857', '865', '885', '888', '895', '928', '931', '932', '934', '936', '939', '942', '943', '945', '946', '947', '948', '950', '951', '952', '957', '958', '959', '960', '961', '963', '964', '967', '968', '969', '970', '971', '972', '973', '974', '975', '977', '978', '980', '982', '984', '985', '987', '988', '990', '992', '994', '995', '996', '997', '998', '999', '1000', '1001', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1019', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1029', '1032', '1038', '1039', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '1152', '1161', '1189', '1193', '1199']}

In [22]:
for x in overlapping.items():
    print(len(x[1]))

48
46
74
167


In [23]:
(48+46+74+167) # 335

335

In [ ]:
{'datalot1': ['151', '185', '261', '307', '317', '321', '324', '461', '514', '518', '519', '520', '522', '524', '525', '526', '527', '528', '532', '533', '535', '554', '556', '558', '559', '561', '563', '564', '573', '574', '575', '577', '581', '624', '625', '643', '656', '672', '683', '687', '690', '695', '701', '711', '714', '723', '726', '791'],
 'datalot2': ['12', '17', '39', '51', '55', '74', '76', '77', '78', '81', '92', '124', '126', '134', '138', '151', '152', '153', '157', '191', '198', '215', '219', '237', '282', '308', '318', '350', '388', '391', '393', '394', '400', '403', '407', '410', '411', '415', '416', '420', '425', '428', '464', '475', '506', '593'], 
 'datalot3': ['23', '25', '49', '65', '67', '110', '116', '118', '120', '125', '138', '166', '183', '184', '207', '248', '249', '250', '255', '263', '274', '275', '278', '279', '280', '284', '286', '323', '325', '336', '339', '348', '388', '389', '391', '417', '418', '419', '421', '423', '424', '425', '427', '428', '429', '432', '433', '434', '435', '437', '440', '443', '444', '445', '446', '448', '449', '455', '467', '477', '495', '499', '500', '509', '511', '514', '519', '524', '542', '544', '556', '560', '568', '572'],
 'datalot4': ['10', '65', '74', '80', '84', '92', '98', '99', '100', '101', '110', '155', '169', '171', '188', '243', '249', '251', '258', '264', '265', '288', '435', '437', '438', '467', '491', '505', '516', '569', '577', '578', '601', '619', '620', '621', '624', '640', '651', '658', '666', '667', '668', '726', '734', '735', '740', '752', '812', '813', '817', '818', '819', '821', '822', '833', '841', '843', '844', '849', '857', '865', '885', '888', '895', '928', '931', '932', '934', '936', '939', '942', '943', '945', '946', '947', '948', '950', '951', '952', '957', '958', '959', '960', '961', '963', '964', '967', '968', '969', '970', '971', '972', '973', '974', '975', '977', '978', '980', '982', '984', '985', '987', '988', '990', '992', '994', '995', '996', '997', '998', '999', '1000', '1001', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1019', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1029', '1032', '1038', '1039', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '1152', '1161', '1189', '1193', '1199']}